In [1]:
pwd

'C:\\Users\\saike'

In [4]:
import os
os.chdir("C:\\Users\\saike\\OneDrive\\Desktop\\Series\\ML\\8. Adaboost")

In [5]:
pwd

'C:\\Users\\saike\\OneDrive\\Desktop\\Series\\ML\\8. Adaboost'

In [92]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as ax
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [7]:
df = pd.read_csv('cardekho_dataset.csv')

In [8]:
df.head()

,Unnamed: 0,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [9]:
df.drop('Unnamed: 0', axis=1, inplace = True)

In [43]:
df.drop(columns = ['brand','model'], axis=1, inplace = True)

In [44]:
df.head()

,car_name,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15411 entries, 0 to 15410
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   car_name           15411 non-null  object 
 1   vehicle_age        15411 non-null  int64  
 2   km_driven          15411 non-null  int64  
 3   seller_type        15411 non-null  object 
 4   fuel_type          15411 non-null  object 
 5   transmission_type  15411 non-null  object 
 6   mileage            15411 non-null  float64
 7   engine             15411 non-null  int64  
 8   max_power          15411 non-null  float64
 9   seats              15411 non-null  int64  
 10  selling_price      15411 non-null  int64  
dtypes: float64(2), int64(5), object(4)
memory usage: 1.3+ MB


In [46]:
df.select_dtypes(include='O').columns

Index(['car_name', 'seller_type', 'fuel_type', 'transmission_type'], dtype='object')

In [47]:
df.select_dtypes(exclude='O').columns

Index(['vehicle_age', 'km_driven', 'mileage', 'engine', 'max_power', 'seats',
       'selling_price'],
      dtype='object')

In [48]:
df.isnull().sum()

car_name             0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [49]:
df.corr()


,vehicle_age,km_driven,mileage,engine,max_power,seats,selling_price
vehicle_age,1.000000,0.333891,-0.257394,0.098965,0.005208,0.030791,-0.241851
km_driven,0.333891,1.000000,-0.105239,0.192885,0.044421,0.192830,-0.080030
mileage,-0.257394,-0.105239,1.000000,-0.632987,-0.533128,-0.440280,-0.305549
engine,0.098965,0.192885,-0.632987,1.000000,0.807368,0.551236,0.585844
max_power,0.005208,0.044421,-0.533128,0.807368,1.000000,0.172257,0.750236
seats,0.030791,0.192830,-0.440280,0.551236,0.172257,1.000000,0.115033
selling_price,-0.241851,-0.080030,-0.305549,0.585844,0.750236,0.115033,1.000000


In [50]:
num_features = [features for features in df.columns if df[features].dtype != 'O']
cat_features = [features for features in df.columns if df[features].dtype == 'O']
dis_features = [features for features in num_features if len(df[features].unique()) <= 25 ]
cont_features = [features for features in num_features if len(df[features].unique()) > 25 ]

In [51]:
print('num_features:', len(num_features))
print('cat_features:', len(cat_features))
print('dis_features:', len(dis_features))
print('cont_features:', len(cont_features))

num_features: 7
cat_features: 4
dis_features: 2
cont_features: 5


In [52]:
type(num_features)

list

In [53]:
X = df.drop('selling_price', axis=1)
y = df.selling_price

In [54]:
X

,car_name,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,Maruti Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,Hyundai Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,Hyundai i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,Maruti Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,Ford Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5
...,...,...,...,...,...,...,...,...,...,...
15406,Hyundai i10,9,10723,Dealer,Petrol,Manual,19.81,1086,68.05,5
15407,Maruti Ertiga,2,18000,Dealer,Petrol,Manual,17.50,1373,91.10,7
15408,Skoda Rapid,6,67000,Dealer,Diesel,Manual,21.14,1498,103.52,5
15409,Mahindra XUV500,5,3800000,Dealer,Diesel,Manual,16.00,2179,140.00,7


In [55]:
y

0         120000
1         550000
2         215000
3         226000
4         570000
          ...   
15406     250000
15407     925000
15408     425000
15409    1225000
15410    1200000
Name: selling_price, Length: 15411, dtype: int64

In [57]:
X.select_dtypes(include='O').columns

Index(['car_name', 'seller_type', 'fuel_type', 'transmission_type'], dtype='object')

In [69]:
col = ['car_name', 'seller_type', 'fuel_type', 'transmission_type']
for c in col:
    
    print(X[c].value_counts())
    print('--------------------------')

Hyundai i20           906
Maruti Swift Dzire    890
Maruti Swift          781
Maruti Alto           778
Honda City            757
                     ... 
Mercedes-AMG C          1
Tata Altroz             1
Ferrari GTC4Lusso       1
Hyundai Aura            1
Force Gurkha            1
Name: car_name, Length: 121, dtype: int64
--------------------------
Dealer              9539
Individual          5699
Trustmark Dealer     173
Name: seller_type, dtype: int64
--------------------------
Petrol      7643
Diesel      7419
CNG          301
LPG           44
Electric       4
Name: fuel_type, dtype: int64
--------------------------
Manual       12225
Automatic     3186
Name: transmission_type, dtype: int64
--------------------------


In [75]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X['car_name'] = le.fit_transform(X['car_name'])
X['fuel_type'] = le.fit_transform(X['fuel_type'])

In [76]:
X

,car_name,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,65,9,120000,Individual,4,Manual,19.70,796,46.30,5
1,34,5,20000,Individual,4,Manual,18.90,1197,82.00,5
2,40,11,60000,Individual,4,Manual,17.00,1197,80.00,5
3,65,9,37000,Individual,4,Manual,20.92,998,67.10,5
4,20,6,30000,Dealer,1,Manual,22.77,1498,98.59,5
...,...,...,...,...,...,...,...,...,...,...
15406,39,9,10723,Dealer,4,Manual,19.81,1086,68.05,5
15407,73,2,18000,Dealer,4,Manual,17.50,1373,91.10,7
15408,101,6,67000,Dealer,1,Manual,21.14,1498,103.52,5
15409,64,5,3800000,Dealer,1,Manual,16.00,2179,140.00,7


In [87]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

one_hot = ['seller_type','transmission_type']
num_col = X.select_dtypes(exclude='O').columns

ct = ColumnTransformer( transformers = [
    ('onehot', OneHotEncoder(),  one_hot),
    ('ss', StandardScaler(), num_col)
], remainder = 'passthrough')


In [88]:
X=ct.fit_transform(X)

In [94]:

import pandas as pd
X = pd.DataFrame(X)


In [95]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state = 40)

In [96]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12
12149,1.0,0.0,0.0,0.0,1.0,0.137617,-0.675807,-0.169431,1.001423,1.198440,-1.324259,-1.240081,-0.403022
11219,1.0,0.0,0.0,0.0,1.0,-1.103489,-1.007681,-0.283173,1.001423,-0.647583,0.020999,0.405192,-0.403022
6947,0.0,1.0,0.0,0.0,1.0,-1.103489,-0.675807,0.472394,-0.967649,1.510106,0.022918,-0.046269,-0.403022
1091,0.0,1.0,0.0,0.0,1.0,-1.103489,1.647309,0.472394,1.001423,-0.647583,0.020999,0.405192,-0.403022
5287,1.0,0.0,0.0,0.0,1.0,0.594867,0.651688,0.104298,1.001423,-0.192071,-0.936610,-0.779312,-0.403022
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11256,1.0,0.0,0.0,0.0,1.0,0.594867,1.979183,0.084924,1.001423,0.500787,-0.936610,-0.780475,-0.403022
14501,1.0,0.0,0.0,0.0,1.0,0.496885,-0.343933,0.046177,1.001423,0.167544,-0.554718,-0.437225,-0.403022
14343,0.0,1.0,0.0,0.0,1.0,-0.744221,0.651688,0.472394,1.001423,-0.645185,0.201390,0.481987,-0.403022
14555,1.0,0.0,0.0,0.0,1.0,1.770651,-1.007681,-0.767510,1.001423,-0.839377,-0.550880,-0.618740,-0.403022


In [97]:
X_test

,0,1,2,3,4,5,6,7,8,9,10,11,12
14354,0.0,1.0,0.0,0.0,1.0,-0.123668,0.651688,1.247335,-0.967649,-0.896916,1.989949,-0.895668,2.073444
145,0.0,1.0,0.0,0.0,1.0,-0.744221,-0.012060,-0.901187,1.001423,-0.645185,0.201390,0.481987,-0.403022
15087,1.0,0.0,0.0,0.0,1.0,-0.744221,0.983562,-0.108811,-0.967649,0.627851,0.184118,0.598343,-0.403022
11614,1.0,0.0,0.0,0.0,1.0,1.476705,-0.675807,-0.283173,-0.967649,0.431262,0.020999,0.184116,-0.403022
15176,1.0,0.0,0.0,0.0,1.0,0.104957,-0.675807,-0.593149,-0.967649,-0.887326,1.329794,0.917158,2.073444
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13528,0.0,1.0,0.0,1.0,0.0,0.202939,-0.675807,-0.141746,1.001423,0.814851,-0.936610,-0.780708,-0.403022
407,1.0,0.0,0.0,0.0,1.0,0.562206,-0.343933,0.181792,-0.967649,1.102543,-0.456846,-0.281308,-0.403022
8120,1.0,0.0,0.0,1.0,0.0,0.006975,-0.343933,0.456896,-0.967649,-1.031172,1.329794,0.451734,2.073444
1442,0.0,1.0,0.0,0.0,1.0,-1.332113,-0.343933,-0.302546,-0.967649,0.735736,0.022918,-0.046502,-0.403022


In [101]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [116]:
def evaluate(true_val, predict_val):
    mse = mean_squared_error(true_val, predict_val)
    mae = mean_absolute_error(true_val, predict_val)
    r2 = r2_score(true_val, predict_val)
    return mse, mae, r2

In [119]:
models = {
    'LR' : LogisticRegression(),
    'SVR' : SVR(),
    'KNR' : KNeighborsRegressor(),
    'DTR' : DecisionTreeRegressor(),
    'RFR' : RandomForestRegressor(),
    'ABR' : AdaBoostRegressor()
}

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    y_train_predict = model.predict(X_train)
    y_test_predict = model.predict(X_test)

    train_mse, train_mae, train_r2 = evaluate(y_train, y_train_predict)
    test_mse, test_mae, test_r2 = evaluate(y_test, y_test_predict)

    print(list(models.keys())[i])

    print(".................................................")
    print('Model Performance for Training set')
    print("MSE: {:.4f}" .format(train_mse))
    print("MAE: {:.4f}" .format(train_mae))
    print("R2score: {:.4f}" .format(train_r2))


    print(".................................................")

    print(".................................................")
    print('Model Performance for Test set')
    print("MSE: {:.4f}" .format(test_mse))
    print("MAE: {:.4f}" .format(test_mae))
    print("R2score: {:.4f}" .format(test_r2))




LR
.................................................
Model Performance for Training set
MSE: 213949839087.0883
MAE: 199337.1897
R2score: 0.7459
.................................................
.................................................
Model Performance for Test set
MSE: 202581833603.6328
MAE: 200782.0305
R2score: 0.6783
SVR
.................................................
Model Performance for Training set
MSE: 892056413141.6870
MAE: 402501.8652
R2score: -0.0597
.................................................
.................................................
Model Performance for Test set
MSE: 677760227087.6368
MAE: 391738.0325
R2score: -0.0762
KNR
.................................................
Model Performance for Training set
MSE: 99149882299.7952
MAE: 93737.6115
R2score: 0.8822
.................................................
.................................................
Model Performance for Test set
MSE: 63783829270.3941
MAE: 113829.5086
R2score: 0.8987
DTR
..

In [120]:
#Hyper parameter tuning
knn_params = {
    "n_neighbors":[5,7,9,10],
    "weights": ['uniform', 'distance'],
    "algorithm": ['auto', 'ball_tree', 'kd_tree', 'brute']
}
rf_params = {"max_depth": [5,8,15,10],
            "max_features": [5,7,"auto",8],
            "min_samples_split": [2,8,15,20],
            "n_estimators": [100,150,200,500,1000]


}
ad_params ={
    "n_estimators":[50,100,125,150],
    "learning_rate": [1.0,10.0,50],
    "loss":['linear', 'square', 'exponential']
}

In [121]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
cv=StratifiedKFold()

for name, model, params in random_cv:
    randomcv_model = RandomizedCV(extimator = model, params_distribution = params, cv=5, n_iter=50, verbose=3 , scoring='r2')
    randomcv_model.fit(X_train, y_train)
    model_params[name]=randomcv_model.best_params_

for params in model_params:
    print("---------Best params for {params}-----------------")
    print(model_params[params])

NameError: name 'random_cv' is not defined

In [ ]:
models = {

    "KNeighborsRegressor":KNeighborsRegressor(weights =  'distance', n_neighbors =  7, algorithm =  'brute'),
    "RandomForestRegressor":RandomForestRegressor(n_estimators = 100, min_samples_split= 8, max_features= 7, max_depth= 15),
    "AdaBoostRegressor":AdaBoostRegressor(n_estimators =  50, loss =  'linear', learning_rate=1.0)
}


for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    print("------------------My Report-----------------------")

    train_mse, train_mae, train_r2 = evaluation (y_train, y_train_pred)
    test_mse, test_mae, test_r2 = evaluation (y_test, y_test_pred)

    print(list(models.keys())[i])

    print(".................................................")
    print('Model Performance for Training set')
    print("MSE: {:.4f}" .format(train_mse))
    print("MAE: {:.4f}" .format(train_mae))
    print("R2score: {:.4f}" .format(train_r2))


    print(".................................................")

    print(".................................................")
    print('Model Performance for Test set')
    print("MSE: {:.4f}" .format(test_mse))
    print("MAE: {:.4f}" .format(test_mae))
    print("R2score: {:.4f}" .format(test_r2))


    print('='*35)
    print('\n')
